# 🤖 TARS v3 — Google Colab Training

**Полное обучение TARS v3 на GPU T4 (16 GB VRAM)**

### Инструкции:
1. **Runtime → Change runtime type → GPU (T4)**
2. Запускай ячейки по порядку ▶
3. В конце скачай обученные модели

---

## 0. Проверка GPU

In [ ]:
import torch
print("=" * 60)
print("  TARS v3 — Google Colab Training")
print("=" * 60)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_mem / 1024**3
    print(f"  ✅ GPU: {gpu_name}")
    print(f"  ✅ VRAM: {vram:.1f} GB")
    print(f"  ✅ CUDA: {torch.version.cuda}")
else:
    print("  ❌ GPU НЕ ОБНАРУЖЕН!")
    print("  → Runtime → Change runtime type → GPU (T4)")
    raise RuntimeError("GPU not found! Enable GPU in Runtime settings.")

print("=" * 60)

## 1. Клонирование репозитория с GitHub

In [ ]:
import os

REPO_URL = "https://github.com/Vazilll/TarsSSM-Py.git"
WORK_DIR = "/content/TarsSSM-Py"

if os.path.exists(os.path.join(WORK_DIR, "mega_train.py")):
    print("✅ Репозиторий уже клонирован")
else:
    print(f"📥 Клонирование {REPO_URL} ...")
    !git clone {REPO_URL} {WORK_DIR}
    print("✅ Клонирование завершено")

os.chdir(WORK_DIR)
print(f"📂 Рабочая директория: {os.getcwd()}")

## 2. Установка зависимостей

In [ ]:
# Torch уже установлен в Colab, ставим остальное
!pip install -q einops tqdm psutil sentencepiece tokenizers \
    sentence-transformers datasets transformers peft jiwer
print("\n✅ Зависимости установлены")

## 3. Скачивание обучающих данных

In [ ]:
import sys
sys.path.insert(0, WORK_DIR)

os.makedirs("data", exist_ok=True)

# Wikipedia (10к статей)
wiki_path = os.path.join(WORK_DIR, "data", "wiki_ru.txt")
if os.path.exists(wiki_path) and os.path.getsize(wiki_path) > 100_000:
    size_mb = os.path.getsize(wiki_path) / 1024 / 1024
    print(f"📚 Wikipedia: уже есть ({size_mb:.1f} MB)")
else:
    print("📚 Скачивание Wikipedia (10 000 статей)...")
    !python training/download_wiki.py --count 10000

# HuggingFace datasets
import glob
hf_files = glob.glob(os.path.join(WORK_DIR, "data", "hf_*.txt"))
if len(hf_files) >= 1:
    total_mb = sum(os.path.getsize(f) for f in hf_files) / 1024 / 1024
    print(f"🤗 HuggingFace: уже есть ({len(hf_files)} файлов, {total_mb:.0f} MB)")
else:
    print("🤗 Скачивание HuggingFace датасетов...")
    !python training/download_hf_dataset.py --preset all

# Итого
data_files = [f for f in glob.glob(os.path.join(WORK_DIR, "data", "*")) if os.path.isfile(f)]
total = sum(os.path.getsize(f) for f in data_files)
print(f"\n📊 Итого данных: {total / 1024 / 1024:.0f} MB")

## 4. Обучение — Фаза 2: Рефлексы (MinGRU Classifier)
~1-2 мин на GPU

In [ ]:
print("=" * 60)
print("  Фаза 2: Рефлексы (MinGRU Classifier)")
print("=" * 60)
!python training/train_reflex.py --epochs 100 --lr 0.002

## 5. Обучение — Фаза 3: MinGRU Language Model (System 1)
~15 мин на T4

In [ ]:
print("=" * 60)
print("  Фаза 3: MinGRU Language Model (System 1)")
print("=" * 60)
!python training/train_mingru.py \
    --epochs 25 \
    --lr 3e-3 \
    --dim 512 \
    --layers 6 \
    --batch 32 \
    --seq_len 256 \
    --augment

## 6. Обучение — Фаза 4: Mamba-2 Brain (основное)

**Это самая долгая фаза (~2-4 часа на T4)**

12 слоёв × 768d, 4 под-фазы:
- Phase 1: Full pretrain (5 эпох)
- Phase 2: Fine-tune WKV + Fusion (3 эпохи)
- Phase 3: Fine-tune MoLE + MatrixPool (2 эпохи)
- Phase 4: Fine-tune WKV + RAG + Memory (2 эпохи)

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
vram_gb = torch.cuda.get_device_properties(0).total_mem / 1024**3 if torch.cuda.is_available() else 0

# T4 = 16GB → batch 16, accum 4
if vram_gb >= 40:
    BATCH, ACCUM = "32", "2"
elif vram_gb >= 16:
    BATCH, ACCUM = "16", "4"
elif vram_gb >= 8:
    BATCH, ACCUM = "8", "8"
else:
    BATCH, ACCUM = "4", "4"

print(f"GPU: {torch.cuda.get_device_name(0)} ({vram_gb:.0f} GB)")
print(f"Batch: {BATCH} × {ACCUM} = {int(BATCH) * int(ACCUM)} effective")

# Transfer embedding from MinGRU
emb_args = ""
mingru_path = os.path.join(WORK_DIR, "models", "mingru_weights.pt")
if os.path.exists(mingru_path):
    cp = torch.load(mingru_path, map_location='cpu', weights_only=False)
    state = cp.get('model_state_dict', cp)
    for k in state:
        if 'shared_embedding' in k or 'emb.weight' in k:
            emb_dir = os.path.join(WORK_DIR, "models", "tars_v3")
            os.makedirs(emb_dir, exist_ok=True)
            emb_path = os.path.join(emb_dir, "_transfer_embedding.pt")
            torch.save(state[k], emb_path)
            emb_args = f"--pretrained_emb {emb_path}"
            print(f"🔗 Transfer embedding: {state[k].shape}")
            break

BASE_ARGS = f"--d_model 768 --n_layers 12 --vocab_size 256 --batch {BATCH} --accum_steps {ACCUM} --device {device} --curriculum --label_smoothing 0.1 {emb_args}"

In [ ]:
# Phase 1/4: Full pretrain
print("=" * 60)
print("  Phase 1/4: Full pretrain (SSD + WKV + Ω-SSM + MoLE)")
print("=" * 60)
!python training/train_mamba2.py {BASE_ARGS} --epochs 5 --lr 3e-4 --phase 1 --seq_len 256

In [ ]:
# Phase 2/4: Fine-tune WKV + Fusion
print("=" * 60)
print("  Phase 2/4: Fine-tune WKV + Fusion (SSD frozen)")
print("=" * 60)
!python training/train_mamba2.py {BASE_ARGS} --epochs 3 --lr 1e-4 --phase 2 --seq_len 512 --resume

In [ ]:
# Phase 3/4: Fine-tune MoLE + MatrixPool
print("=" * 60)
print("  Phase 3/4: Fine-tune MoLE + MatrixPool")
print("=" * 60)
!python training/train_mamba2.py {BASE_ARGS} --epochs 2 --lr 3e-5 --phase 3 --seq_len 512 --resume

In [ ]:
# Phase 4/4: Fine-tune WKV + RAG + Memory
print("=" * 60)
print("  Phase 4/4: Fine-tune WKV + RAG + Memory")
print("=" * 60)
!python training/train_mamba2.py {BASE_ARGS} --epochs 2 --lr 1.5e-5 --phase 4 --seq_len 512 --resume

## 7. Квантизация BitNet 1.58-bit
~15 мин на T4

In [ ]:
fp16_path = os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega.pt")

if os.path.exists(fp16_path):
    print("=" * 60)
    print("  Фаза 5: Квантизация BitNet 1.58-bit")
    print("=" * 60)
    !python training/train_mamba2.py \
        --d_model 768 --n_layers 12 \
        --batch 16 --accum_steps 4 \
        --epochs 3 --lr 5e-5 \
        --phase 1 --quant \
        --resume --device cuda \
        --seq_len 256 --label_smoothing 0.1
else:
    print("⚠ FP16 модель не найдена, пропускаем квантизацию")

## 8. Валидация модели

In [ ]:
import torch
import sys
sys.path.insert(0, WORK_DIR)

from brain.tokenizer import TarsTokenizer
from brain.mamba2.model import TarsMamba2LM

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = TarsTokenizer()
model, ckpt = TarsMamba2LM.load_pretrained(device=device)
model.eval()

if ckpt is None:
    print("⚠ Trained weights not found")
else:
    params = sum(p.numel() for p in model.parameters())
    print(f"✅ Model: {ckpt}")
    print(f"✅ Parameters: {params:,}")
    print()
    
    for prompt in ["привет", "как дела", "что такое", "расскажи"]:
        tokens = tokenizer.encode(prompt)
        input_ids = torch.tensor([tokens], dtype=torch.long, device=device)
        with torch.no_grad():
            logits = model(input_ids)
        probs = torch.softmax(logits[0, -1, :], dim=-1)
        top5 = torch.topk(probs, 5)
        preds = []
        for idx, prob in zip(top5.indices.tolist(), top5.values.tolist()):
            char = tokenizer.decode([idx])
            preds.append(f"'{char}'({prob:.2%})")
        print(f"  '{prompt}' → {', '.join(preds)}")
    
    print("\n✅ Model works!")

## 9. 💾 Сохранение и скачивание моделей

Результаты будут заархивированы в `/content/tars_v3_models.tar.gz`

In [ ]:
import shutil
import json

OUTPUT_DIR = "/content/tars_v3_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Файлы моделей
model_files = {
    "mamba2.pt":        os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega.pt"),
    "mamba2_158bit.pt": os.path.join(WORK_DIR, "models", "mamba2", "mamba2_omega_158bit.pt"),
    "mingru.pt":        os.path.join(WORK_DIR, "models", "mingru_weights.pt"),
    "reflex.pt":        os.path.join(WORK_DIR, "models", "reflex", "reflex_classifier.pt"),
}

print("📦 Сохранение моделей:")
for name, src in model_files.items():
    if os.path.exists(src):
        dst = os.path.join(OUTPUT_DIR, name)
        shutil.copy2(src, dst)
        size_mb = os.path.getsize(dst) / 1024 / 1024
        print(f"  ✅ {name} ({size_mb:.1f} MB)")
    else:
        print(f"  ⏭ {name} — не найден")

# Config
config = {
    "models": {
        "mamba2": {
            "params": {
                "encoding": "cp1251",
                "vocab_size": 256,
                "d_model": 768,
                "n_layers": 12,
                "n_experts": 8,
                "omega_dim": 32,
                "pool_size": 48
            }
        }
    }
}
with open(os.path.join(OUTPUT_DIR, "config.json"), "w") as f:
    json.dump(config, f, indent=2)

# Архивируем
archive = "/content/tars_v3_models"
shutil.make_archive(archive, 'gztar', OUTPUT_DIR)
archive_path = archive + ".tar.gz"
size_mb = os.path.getsize(archive_path) / 1024 / 1024
print(f"\n📁 Архив: {archive_path} ({size_mb:.1f} MB)")

In [ ]:
# Скачать архив с моделями
from google.colab import files
files.download("/content/tars_v3_models.tar.gz")
print("\n🎯 Скачивание начато!")

## 🔄 (Опционально) Сохранение на Google Drive

Если сессия может отключиться — сохрани промежуточные результаты на Drive

In [ ]:
# Раскомментируй если хочешь сохранить на Google Drive

# from google.colab import drive
# drive.mount('/content/drive')
#
# import shutil
# drive_dir = "/content/drive/MyDrive/TARS_models"
# os.makedirs(drive_dir, exist_ok=True)
#
# for name, src in model_files.items():
#     if os.path.exists(os.path.join(OUTPUT_DIR, name)):
#         shutil.copy2(os.path.join(OUTPUT_DIR, name), os.path.join(drive_dir, name))
#         print(f"  💾 {name} → Google Drive")
#
# print("✅ Сохранено на Google Drive!")

---

### ⏱ Оценка времени по фазам (T4 16GB):

| Фаза | Описание | Время |
|------|----------|-------|
| 0 | Установка зависимостей | ~2 мин |
| 1 | Скачивание данных | ~10 мин |
| 2 | Рефлексы | ~2 мин |
| 3 | MinGRU LM | ~15 мин |
| 4 | Mamba-2 (4 под-фазы) | ~2-4 часа |
| 5 | Квантизация | ~15 мин |
| 7 | Валидация | ~1 мин |
| **Итого** | | **~3-5 часов** |